# Rapid API post

- title: Rapid API post
- toc: true 
- badges: true
- comments: true
- categories: [jupyter]

In [4]:
import java.net.URI;
import java.net.http.HttpClient;
import java.net.http.HttpRequest;
import java.net.http.HttpResponse;

//RapidAPI header  https://rapidapi.com/Coinranking/api/coinranking1/
HttpRequest request = HttpRequest.newBuilder()
		.uri(URI.create("https://coinranking1.p.rapidapi.com/coin/Qwsogvtv82FCd/markets?referenceCurrencyUuid=yhjMzLPhuIDl&limit=50&offset=0&orderBy=24hVolume&orderDirection=desc"))
		.header("X-RapidAPI-Key", "3bef89a477msh7eecb5e7498cd51p1095f2jsn9b36f511d7ec")
		.header("X-RapidAPI-Host", "coinranking1.p.rapidapi.com")
		.method("GET", HttpRequest.BodyPublishers.noBody())
		.build();
HttpResponse<String> response = HttpClient.newHttpClient().send(request, HttpResponse.BodyHandlers.ofString());
System.out.println(response.body());

{"status":"success","data":{"stats":{"24hVolume":"50213353283.55389","total":5089},"markets":[{"uuid":"MP77r-vKf4","rank":1,"base":{"uuid":"Qwsogvtv82FCd","symbol":"BTC"},"quote":{"uuid":"HIVsRcGKkPFtW","symbol":"USDT"},"exchange":{"name":"Binance","uuid":"-zdvbieRdZ","iconUrl":"https://cdn.coinranking.com/mDTK5qrmq/binance.svg"},"marketShare":"21.02","btcPrice":"0.9997433437240185","recommended":true,"filters":[],"price":"19544.769220336657","24hVolume":"10553015522.155455"},{"uuid":"ifeyn_cix4","rank":2,"base":{"uuid":"Qwsogvtv82FCd","symbol":"BTC"},"quote":{"uuid":"vSo2fu9iE1s0Y","symbol":"BUSD"},"exchange":{"name":"Binance","uuid":"-zdvbieRdZ","iconUrl":"https://cdn.coinranking.com/mDTK5qrmq/binance.svg"},"marketShare":"8.23","btcPrice":"0.9999039376203765","recommended":true,"filters":[],"price":"19547.908796771168","24hVolume":"4134659012.21242"},{"uuid":"oZxYFuELeuV","rank":3,"base":{"uuid":"Qwsogvtv82FCd","symbol":"BTC"},"quote":{"uuid":"HIVsRcGKkPFtW","symbol":"USDT"},"exchang

### Build your own Web Service
> To build a Web Service in the backend allows you to control information and errors between your app and the user.  A proper Web Service requires more code: RESTful interface, error handling, and in this case reducing the calls to the service according to update frequency.
- There are benefits of building service in Java vs JavaScript:
    - Firewall will not be blocked as backend server on AWS is outside the firewall
    - Selecting frontend service https://nighthawkcodingsociety.com that has been cleared through IT.
    - Many APIs have usage costs or usage limits, backend can serve while limiting direct API access.
- Project link: https://github.com/nighthawkcoders/spring_portfolio/blob/master/src/main/java/com/nighthawk/spring_portfolio/mvc/covid/CovidApiController.java

```java
package com.nighthawk.spring_portfolio.mvc.covid;

import java.net.URI;
import java.net.http.HttpClient;
import java.net.http.HttpRequest;
import java.net.http.HttpResponse;
import java.util.Date;
import java.util.HashMap;

import org.json.simple.JSONObject;
import org.json.simple.parser.JSONParser;
import org.springframework.http.HttpStatus;
import org.springframework.http.ResponseEntity;
import org.springframework.web.bind.annotation.*;

@RestController // annotation to create a RESTful web services
@RequestMapping("/api/covid")  //prefix of API
public class CovidApiController {
    private JSONObject body; //last run result
    private HttpStatus status; //last run status
    String last_run = null; //last run day of month

    // GET Covid 19 Stats
    @GetMapping("/daily")   //added to end of prefix as endpoint
    public ResponseEntity<JSONObject> getCovid() {

        //calls API once a day, sets body and status properties
        String today = new Date().toString().substring(0,10); 
        if (last_run == null || !today.equals(last_run))
        {
            try {  //APIs can fail (ie Internet or Service down)
                
                //RapidAPI header
                HttpRequest request = HttpRequest.newBuilder()
                    .uri(URI.create("https://corona-virus-world-and-india-data.p.rapidapi.com/api"))
                    .header("x-rapidapi-key", "dec069b877msh0d9d0827664078cp1a18fajsn2afac35ae063")
                    .header("x-rapidapi-host", "corona-virus-world-and-india-data.p.rapidapi.com")
                    .method("GET", HttpRequest.BodyPublishers.noBody())
                    .build();

                //RapidAPI request and response
                HttpResponse<String> response = HttpClient.newHttpClient().send(request, HttpResponse.BodyHandlers.ofString());

                //JSONParser extracts text body and parses to JSONObject
                this.body = (JSONObject) new JSONParser().parse(response.body());
                this.status = HttpStatus.OK;  //200 success
                this.last_run = today;
            }
            catch (Exception e) {  //capture failure info
                HashMap<String, String> status = new HashMap<>();
                status.put("status", "RapidApi failure: " + e);

                //Setup object for error
                this.body = (JSONObject) status;
                this.status = HttpStatus.INTERNAL_SERVER_ERROR; //500 error
                this.last_run = null;
            }
        }

        //return JSONObject in RESTful style
        return new ResponseEntity<>(body, status);
    }
}
```